# 06 Worst-case Report

Rank worst runs by tail latency and export a summary table.


In [1]:
from pathlib import Path
import os
import pandas as pd
import numpy as np

ANALYZE_DIR = Path('../outputFiles/analyze').resolve()
REPORT_PREFIX = os.environ.get('REPORT_PREFIX', 'analysis_reports')
REPORT_DIR = (Path('../outputFiles/analyze').resolve() / REPORT_PREFIX)
STATS_CSV = None  # set to a specific file path if needed
TOPK_CSV = None   # set to a specific file path if needed

def pick_latest(pattern):
    files = sorted(ANALYZE_DIR.glob(pattern))
    if not files:
        raise FileNotFoundError(f'No files matched: {pattern}')
    return files[-1]

stats_path = Path(STATS_CSV) if STATS_CSV else pick_latest('collected_stats_*.csv')
topk_path = Path(TOPK_CSV) if TOPK_CSV else pick_latest('collected_topk_*.csv')

print('stats:', stats_path)
print('topk :', topk_path)

stats_df = pd.read_csv(stats_path)
topk_df = pd.read_csv(topk_path)


stats: /home/gt/research/DiskANN/scripts/paramAnalysis/gridSearch/outputFiles/analyze/collected_stats_exp01_20260101_235226.csv
topk : /home/gt/research/DiskANN/scripts/paramAnalysis/gridSearch/outputFiles/analyze/collected_topk_exp01_20260101_235226.csv


In [2]:
out_tables = (REPORT_DIR / 'tables')
out_tables.mkdir(parents=True, exist_ok=True)

cols = [
    'run_prefix','recall_mean','latency_p99_us','latency_p999_us',
    'ios_p99','queue_depth_p99','io_us_p99','cpu_us_p99',
    'thread_util_p99','expanded_revisit_ratio','node_counts_top10_share',
]
cols = [c for c in cols if c in stats_df.columns]
worst = stats_df.sort_values('latency_p99_us', ascending=False).head(20)[cols]
worst.to_csv(out_tables / 'worst_runs.csv', index=False)
worst


,run_prefix,recall_mean,latency_p99_us,latency_p999_us,ios_p99,queue_depth_p99,io_us_p99,cpu_us_p99,thread_util_p99,expanded_revisit_ratio,node_counts_top10_share
64,S4_siftsmall_R64_L256_B0.2_M1_W1_L10_K10_cache...,0.988,2305.7429,2305.7429,13.0,1.0,2265.4409,41.595,0.9905,0.216988,0.123552
13,S3_siftsmall_R16_L32_B0.2_M1_W1_L10_K10_cache0...,0.908,1506.0170,1506.0170,22.0,1.0,1473.0669,32.196,0.9813,0.241751,0.131313
23,S3_siftsmall_R16_L64_B0.2_M1_W1_L10_K10_cache0...,0.924,1458.8800,1458.8800,21.0,1.0,1428.1901,30.598,0.9834,0.251656,0.127815
43,S3_siftsmall_R32_L64_B0.2_M1_W1_L10_K10_cache0...,0.968,1338.7570,1338.7570,18.0,1.0,1301.1501,30.926,0.9815,0.237990,0.134516
53,S3_siftsmall_R64_L128_B0.2_M1_W1_L10_K10_cache...,0.992,1320.3220,1320.3220,17.0,1.0,1253.1770,60.061,0.9759,0.219814,0.123065
19,S9_siftsmall_R16_L32_B0.2_M1_W1_L10_K10_cache2...,0.908,1315.4530,1315.4530,19.0,1.0,1275.1301,25.549,0.9784,0.241751,0.131313
56,S6_siftsmall_R64_L128_B0.2_M1_W1_L10_K10_cache...,0.992,1311.7970,1311.7970,15.0,1.0,1246.6450,58.648,0.9760,0.219814,0.123065
16,S6_siftsmall_R16_L32_B0.2_M1_W1_L10_K10_cache1...,0.908,1310.2640,1310.2640,20.0,1.0,1266.4832,25.327,0.9747,0.241751,0.131313
52,S2_siftsmall_R64_L128_B0.2_M1_W1_L10_K10_cache...,0.992,1304.5830,1304.5830,17.0,1.0,1260.5620,48.702,0.9893,0.219814,0.123065
11,S1_siftsmall_R16_L32_B0.2_M1_W1_L10_K10_cache0_T1,0.908,1291.6390,1291.6390,22.0,1.0,1265.6100,20.208,0.9935,0.241751,0.131313
